In [1]:
import io
import os
import shutil

import numpy as np
import tensorflow as tf

# tf.enable_eager_execution()

In [2]:
filename = '/tmp/data/mnist/tmpbjd0WU/train_shuffled-00000-of-00001.gz'

In [3]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP')

In [4]:
FEATURE_DESCRIPTION = {
    'image': tf.FixedLenFeature([], tf.string, default_value=''),
    'label': tf.FixedLenFeature([], tf.int64, default_value=0),
}

def _parse_function(example_proto):
    # Parse the input tf.Example proto using the dictionary above.
    return tf.parse_single_example(example_proto, FEATURE_DESCRIPTION)

parsed_dataset = raw_dataset.map(_parse_function)

In [5]:
from examples.aaron_simple_example import *

In [6]:
from logzero import logger

# NOTE: 1 epoch for dev / debugging
TRAIN_NUM_EPOCHS = 1

def get_feature_columns():
    image_column = tf.feature_column.numeric_column(IMAGE_KEY, shape=[HEIGHT, WIDTH])
    return [image_column]

def make_input_fn(filename, batch_size):
    def input_fn():
        dataset = tf.data.TFRecordDataset(
            [filename], compression_type='GZIP')
        dataset = dataset.map(decode)
        dataset = dataset.map(augment)
        dataset = dataset.map(normalize)
        dataset = dataset.batch(batch_size)
        steps = NUM_TRAIN_INSTANCES // TRAIN_BATCH_SIZE
        dataset = dataset.repeat(TRAIN_NUM_EPOCHS * steps)
        image, label = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
        return {IMAGE_KEY: image}, label
    return input_fn

estimator = tf.estimator.DNNClassifier(
    feature_columns=get_feature_columns(),
    hidden_units=[784, 10],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
)

# train
train_steps = TRAIN_NUM_EPOCHS * NUM_TRAIN_INSTANCES / TRAIN_BATCH_SIZE
logger.info('train_steps: %s', train_steps)

estimator.train(
    input_fn=make_input_fn(
        filename='/tmp/data/mnist/tmpbjd0WU/train_shuffled-00000-of-00001.gz',
        batch_size=28
    ),
    max_steps=train_steps)

# eval
result = estimator.evaluate(
    input_fn=make_input_fn(
        filename='/tmp/data/mnist/tmpbjd0WU/test_shuffled-00000-of-00001.gz',
        batch_size=1
    ),
    steps=NUM_TEST_INSTANCES)

result

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12bd74cd0>, '_model_dir': '/var/folders/k5/bp3zwqms0bx9sp3p4mfl4vf80000gn/T/tmpCDUJYy', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


[I 190516 07:00:25 <ipython-input-6-ef1662aa23a3>:34] train_steps: 468


Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/k5/bp3zwqms0bx9sp3p4mfl4vf80000gn/T/tmpCDUJYy/model.ckpt.
INFO:tensorflow:loss = 23.909554, step = 1
INFO:tensorflow:global_step/sec: 133.087
INFO:tensorflow:loss = 4.2669134, step = 101 (0.753 sec)
INFO:tensorflow:global_step/sec: 153.482
INFO:tensorflow:loss = 2.777055, step = 201 (0.651 sec)
INFO:tensorflow:global_step/sec: 153.499
INFO:tensorflow:loss = 1.2009966, step = 301 (0.651 sec)
INFO:tensorflow:global_step/sec: 153.39
INFO:tensorflow:loss = 1.2477928, s

{'accuracy': 0.5,
 'average_loss': 1.6720625,
 'global_step': 468,
 'loss': 1.6720625}

### save model

In [7]:
version = 1
export_path = '/tmp/mnist-wed/%s' % version
if os.path.exists(export_path):
    shutil.rmtree(export_path)

logger.info('Exporting trained model to', export_path)
builder = tf.saved_model.builder.SavedModelBuilder(export_path)
builder

[I 190516 07:00:36 <ipython-input-7-81a3ce16356c>:6] Bad message (TypeError('not all arguments converted during string formatting',)): {'threadName': 'MainThread', 'name': 'logzero_default', 'thread': 140735958913920, 'created': 1558015236.241961, 'process': 21737, 'processName': 'MainProcess', 'args': ('/tmp/mnist-wed/1',), 'module': '<ipython-input-7-81a3ce16356c>', 'filename': '<ipython-input-7-81a3ce16356c>', 'levelno': 20, 'exc_text': None, 'pathname': '<ipython-input-7-81a3ce16356c>', 'lineno': 6, 'msg': 'Exporting trained model to', 'exc_info': None, 'funcName': '<module>', 'relativeCreated': 16061.491966247559, 'levelname': 'INFO', 'msecs': 241.96100234985352}


In [8]:
sess = tf.InteractiveSession()

serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
classification_inputs = tf.saved_model.utils.build_tensor_info(
    serialized_tf_example)

prediction_classes = tf.placeholder(tf.string, shape=[None, 10])
classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
    prediction_classes)

prediction_scores = tf.placeholder(tf.float32, shape=[None, 10])
classification_outputs_scores = tf.saved_model.utils.build_tensor_info(
    prediction_scores)

classification_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={
              tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                  classification_inputs
          },
          outputs={
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                  classification_outputs_classes,
              tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                  classification_outputs_scores
          },
          method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

# X - serving input type/shape
# x = tf.placeholder(tf.float32, shape=[HEIGHT, WIDTH])
serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
tf_example = tf.parse_example(serialized_tf_example, feature_configs)
x = tf.identity(tf_example['x'], name='x')
tensor_info_x = tf.saved_model.utils.build_tensor_info(x)

# Y - serving output type/shape
y = tf.placeholder(tf.float32, shape=[None, 10])
tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'images': tensor_info_x},
        outputs={'scores': tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

builder.add_meta_graph_and_variables(
  sess, [tf.saved_model.tag_constants.SERVING],
  signature_def_map={
      'predict_images':
          prediction_signature,
      tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
          classification_signature,
  },
  main_op=tf.tables_initializer(),
  strip_default_attrs=True)

builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/mnist-wed/1/saved_model.pb


'/tmp/mnist-wed/1/saved_model.pb'

### test saved model with `tf.serving` and `docker`